## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [1]:
DATA_URL = "http://lib.ru/LITRA/PUSHKIN/kapitan.txt"

Устанавливаем библиотеки

In [2]:
import warnings
warnings.filterwarnings('ignore')
!pip -q install rnnmorph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 66.1 MB/s eta 0:00:00


Скачиваем текст, по которому будет дано задание, с помощью [urllib](https://docs.python.org/3/library/urllib.request.html)

In [3]:
import urllib.request

opener = urllib.request.URLopener({})

with opener.open(DATA_URL) as resource:
    raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [4]:
raw_text[:200]

'<html><head><title>Александр Сергеевич Пушкин. Капитанская дочка</title></head><body><pre><div align=right><form action=/LITRA/PUSHKIN/kapitan.txt><select name=format><OPTION VALUE="_Contents">Содержа'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки [Beautiful Soap](https://beautiful-soup-4.readthedocs.io/en/latest/)

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

#print(soup.prettify())

In [6]:
# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [7]:
cleaned_text[:200]

'Александр Сергеевич Пушкин. Капитанская дочкаСодержаниеFine HTMLPrinted versiontxt(Word,КПК)Lib.ru html\nАлександр Сергеевич Пушкин. Капитанская дочка\n\n     --------------------------------------------'

С помощью библиотеки [NLTK](https://nltk.readthedocs.io/en/stable/) разбиваем текст на предложения и токены.

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt') # download list of punctuation marks

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"A total of 3395 'sentences'"

Создаём объект морфологического анализатора `RNNMorph`

In [9]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [10]:
from tqdm import tqdm # customisable progressbar decorator for iterators

predictions = [[pred.normal_form for pred in sent if pred.normal_form.isalpha()] for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] # Избавились от токенов типа "точка", "запятая" и т.д.

54/54 [==============================] - 32s 574ms/step


sentences: 100%|██████████| 3395/3395 [00:00<00:00, 168387.18it/s]


[['тот',
  'который',
  'замышлять',
  'у',
  'мы',
  'невозможный',
  'переворот',
  'или',
  'молодой',
  'и',
  'не',
  'знать',
  'наш',
  'народ',
  'или',
  'уж',
  'человек',
  'жестокосердый',
  'кой',
  'чужой',
  'головушка',
  'полушка',
  'да',
  'и',
  'свой',
  'шейка',
  'копейка']]

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 3395 (возможны расхождения в несколько предложений)
*   Токенов: примерно 33199 (возможны расхождения в некоторое количество токенов)

In [11]:
len(predictions)

3395

In [12]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 


33199

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 200 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [13]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords") # download list of stopwords
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

fdist = FreqDist(non_uniq_tokens)
most_common_words = [pair[0] for pair in fdist.most_common(200)]
len([word for word in most_common_words if word not in STOPWORDS]) / 200

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0.66

Проверьте себя: должно получиться 0.66 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 10 раз.

In [14]:
len([word for word in fdist if fdist[word] > 10])

410

Проверьте себя: должно получиться значение 410 (возможно небольшое расхождение)
